# Carregar no PostgreSQL localmente

Esse notebook foi necessário para carregar os dados no Postgres, remover informações que não seriam utilizadas na análise via SQL, e por fim puxá-las de volta localmente salvando assim os dados remanescentes em formato csv para melhor manipulação.

# Inserindo dados de Orders
Foi preciso inserir dados em batch para a tabela orders devio ao alto volume de dados que sobrecarrega a memória.

In [ ]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values

# Conexão com o banco de dados PostgreSQL
conn = psycopg2.connect(
    dbname="iFood",        # Nome do banco de dados
    user="postgres",       # Nome do usuário
    password="1234",       # Senha do usuário
    host="localhost",      # Endereço do servidor (localhost se for local)
    port="5432"            # Porta padrão do PostgreSQL
)
cur = conn.cursor()

# Script para criar a tabela se não existir
create_table_sql = """
CREATE TABLE IF NOT EXISTS arroz (
    cpf TEXT,
    customer_id TEXT,
    customer_name TEXT,
    delivery_address_city TEXT,
    delivery_address_country TEXT,
    delivery_address_district TEXT,
    delivery_address_external_id TEXT,
    delivery_address_latitude TEXT,
    delivery_address_longitude TEXT,
    delivery_address_state TEXT,
    delivery_address_zip_code TEXT,
    items TEXT,
    merchant_id TEXT,
    merchant_latitude TEXT,
    merchant_longitude TEXT,
    merchant_timezone TEXT,
    order_created_at TEXT,
    order_id TEXT,
    order_scheduled TEXT,
    order_total_amount TEXT,
    origin_platform TEXT,
    order_scheduled_date TEXT
);"""

# Executar o script de criação da tabela
cur.execute(create_table_sql)

# Função para inserir os dados em batches
def insert_data_batch(data_batch):
    insert_sql = """
    INSERT INTO arroz (
        cpf, customer_id, customer_name, delivery_address_city, 
        delivery_address_country, delivery_address_district, delivery_address_external_id,
        delivery_address_latitude, delivery_address_longitude, delivery_address_state, 
        delivery_address_zip_code, items, merchant_id, merchant_latitude, 
        merchant_longitude, merchant_timezone, order_created_at, order_id, 
        order_scheduled, order_total_amount, origin_platform, order_scheduled_date
    ) VALUES %s;
    """
    execute_values(cur, insert_sql, data_batch)

# Caminho do arquivo JSON
input_file = '.Files/order.json'

# Processando o arquivo em chunks
chunk_size = 5000  # Número de registros por batch

# Ler o arquivo JSON em pedaços
for chunk in pd.read_json(input_file, lines=True, chunksize=chunk_size):
    data_batch = []
    
    for index, row in chunk.iterrows():
        # Verificar se 'order_scheduled_date' está presente e não é nulo
        order_scheduled_date = row.get('order_scheduled_date', None)  # Usar None se a coluna não existir

        # Preparar os dados para cada linha do dataframe
        order_data = (
            row['cpf'], row['customer_id'], row['customer_name'], row['delivery_address_city'],
            row['delivery_address_country'], row['delivery_address_district'], row['delivery_address_external_id'],
            row['delivery_address_latitude'], row['delivery_address_longitude'], row['delivery_address_state'],
            row['delivery_address_zip_code'], row['items'], row['merchant_id'], row['merchant_latitude'],
            row['merchant_longitude'], row['merchant_timezone'], row['order_created_at'], row['order_id'],
            row['order_scheduled'], row['order_total_amount'], row['origin_platform'], order_scheduled_date
        )
        
        data_batch.append(order_data)

    # Inserir o batch de dados
    insert_data_batch(data_batch)
    conn.commit()
    print(f"Inserted {len(data_batch)} records.")

# Fechar a conexão
cur.close()
conn.close()

print("Dados inseridos com sucesso!")


Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 1000 records.
Inserted 10

# Dados de restaurants

In [ ]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values

# Configurações de conexão com o PostgreSQL
conn = psycopg2.connect(
    dbname="iFood",        # Nome do banco de dados
    user="postgres",       # Nome do usuário
    password="1234",       # Senha do usuário
    host="localhost",      # Endereço do servidor (localhost se for local)
    port="5432"            # Porta padrão do PostgreSQL
)
cur = conn.cursor()

# Criando a tabela 'restaurants' se não existir
create_table_sql = """
CREATE TABLE IF NOT EXISTS restaurant (
    id SERIAL PRIMARY KEY,
    created_at TIMESTAMP,
    enabled BOOLEAN,
    price_range NUMERIC,
    average_ticket NUMERIC,
    takeout_time NUMERIC,
    delivery_time NUMERIC,
    minimum_order_value NUMERIC,
    merchant_zip_code TEXT,
    merchant_city TEXT,
    merchant_state TEXT,
    merchant_country TEXT
);
"""

# Executar o script de criação da tabela
cur.execute(create_table_sql)

# Função para inserir dados de uma vez
def insert_all_data(data_batch):
    insert_sql = """
    INSERT INTO restaurant (
        created_at, enabled, price_range, average_ticket, takeout_time, delivery_time, 
        minimum_order_value, merchant_zip_code, merchant_city, merchant_state, merchant_country
    ) VALUES %s;
    """
    execute_values(cur, insert_sql, data_batch)

# Caminho do arquivo CSV
input_file = './Files/restaurant.csv'

# Ler o arquivo CSV com o pandas
df = pd.read_csv(input_file)

# Preparando os dados para inserção
data_batch = []
for index, row in df.iterrows():
    # Preparar os dados para cada linha do DataFrame
    restaurant_data = (
        row['created_at'], row['enabled'], row['price_range'], row['average_ticket'], 
        row['takeout_time'], row['delivery_time'], row['minimum_order_value'],
        row['merchant_zip_code'], row['merchant_city'], row['merchant_state'], row['merchant_country']
    )
    data_batch.append(restaurant_data)

# Inserir todos os dados de uma vez
insert_all_data(data_batch)
conn.commit()

print(f"Inserted {len(data_batch)} records.")

# Fechar a conexão
cur.close()
conn.close()

print("Dados inseridos com sucesso!")


# Dados de ab_test

In [ ]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values

# Configurações de conexão com o PostgreSQL
conn = psycopg2.connect(
    dbname="iFood",        # Nome do banco de dados
    user="postgres",       # Nome do usuário
    password="1234",       # Senha do usuário
    host="localhost",      # Endereço do servidor (localhost se for local)
    port="5432"            # Porta padrão do PostgreSQL
)
cur = conn.cursor()

# Criando a tabela 'ab_test' se não existir
create_table_sql = """
CREATE TABLE IF NOT EXISTS ab_test (
    customer_id TEXT,
    is_target TEXT
);
"""

# Executar o script de criação da tabela
cur.execute(create_table_sql)

# Função para inserir dados de uma vez
def insert_all_data(data_batch):
    insert_sql = """
    INSERT INTO ab_test (
        customer_id, is_target
    ) VALUES %s;
    """
    execute_values(cur, insert_sql, data_batch)

# Caminho do arquivo CSV
input_file = './Files/ab_test_ref.csv'

# Ler o arquivo CSV com o pandas
df = pd.read_csv(input_file)

# Preparando os dados para inserção
data_batch = []
for index, row in df.iterrows():
    # Preparar os dados para cada linha do DataFrame
    ab_test_data = (
        row['customer_id'], row['is_target']  # Corrigido para usar o nome correto da coluna
    )
    data_batch.append(ab_test_data)

# Inserir todos os dados de uma vez
insert_all_data(data_batch)
conn.commit()

print(f"Inserted {len(data_batch)} records.")

# Fechar a conexão
cur.close()
conn.close()

print("Dados inseridos com sucesso!")
